## Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

* Create a Jupyter Notebook called `database_engineering.ipynb` and use this to complete all of your Database Engineering work.

* Use Pandas to read your cleaned measurements and stations CSV data.

* Use the `engine` and connection string to create a database called `hawaii.sqlite`.

* Use `declarative_base` and create ORM classes for each table.

  * You will need a class for `Measurement` and for `Station`.

  * Make sure to define your primary keys.

* Once you have your ORM classes defined, create the tables in the database using `create_all`.


In [1]:
#Dependencies
import pandas as pd
import numpy as np
# Import SQLAlchemy `automap` and other dependencies
# Import SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, MetaData
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Text, Numeric

In [2]:
measurement = "clean_hawaii_measurements.csv"
station = "clean_hawaii_stations.csv"

df_measurement = pd.read_csv(measurement)
df_station = pd.read_csv(station)

df_station.head()

,Unnamed: 0,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [3]:
# Create the connection engine
engine = create_engine("sqlite:///hawaii.sqlite")

In [4]:
# interface with engine via metadata
Base.metadata.create_all(engine)

In [6]:
engine.table_names()

['clean_measurements', 'clean_stations', 'measurements', 'stations']

In [5]:
#start session
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [7]:
#convert dfs to dicts in order to create classes
measurement = df_measurement.to_dict(orient='records')
station = df_station.to_dict(orient='records')

In [8]:
# Create Measurement and Station classes

Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'
    
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.measurement}"
    
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.station}"


In [9]:
# Create Database Connection
Base.metadata.create_all(engine)

In [10]:
engine.table_names()

['clean_measurements',
 'clean_stations',
 'measurement',
 'measurements',
 'station',
 'stations']

In [11]:
conn = engine.connect()

In [15]:
# avoid duplicate tables
conn.execute(Measurement.__table__.delete())
conn.execute(Station.__table__.delete())
             

In [16]:
# Connect
conn.execute(Measurement.__table__.insert(), measurement)
conn.execute(Station.__table__.insert(), station)

In [17]:
engine.execute("select date from measurement limit 10 ").fetchall()

[('2010-01-01',),
 ('2010-01-02',),
 ('2010-01-03',),
 ('2010-01-04',),
 ('2010-01-07',),
 ('2010-01-08',),
 ('2010-01-09',),
 ('2010-01-10',),
 ('2010-01-11',),
 ('2010-01-12',)]

In [18]:
engine.execute("select * from station limit 1").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0)]

In [13]:
Station.__table__

Table('station', MetaData(bind=None), Column('id', Integer(), table=<station>, primary_key=True, nullable=False), Column('station', Text(), table=<station>), Column('name', Text(), table=<station>), Column('latitude', Float(), table=<station>), Column('longitude', Float(), table=<station>), Column('elevation', Float(), table=<station>), schema=None)